# INF582 AXA Challenge

## Initialisation

In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import itertools
import random

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split
from sklearn import __version__
from sklearn.grid_search import RandomizedSearchCV

pd.set_option('display.max_columns', 500)

## Load data

In [3]:
# Load data
training_data = pd.read_csv("data/train_2011_2012.csv", sep=';', nrows=100000,
                            na_values=['NaN', 'A Définir', 'A DEFINIR', '9999-12-31 00:00:00.000'])
training_data.head()

,DATE,DAY_OFF,DAY_DS,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ACD_COD,ACD_LIB,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_ACWOUTADJCALLS,CSPL_AUXOUTADJCALLS,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_DISCCALLS,CSPL_DISCTIME,CSPL_INTRVL,CSPL_INCOMPLETE,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,855,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1056,1,Entity1 G3RV6,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1057,1,Entity1 G3RV6,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1587,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,30,0,0,15,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1589,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0


## Clean data

In [4]:
# Remove columns containing only one value
for name in training_data.columns:
    count = training_data[name].unique().size
    if count <= 1:
        training_data.drop(name, 1, inplace=True)
        
training_data.head(n=10)

,DATE,WEEK_END,DAY_WE_DS,TPER_HOUR,SPLIT_COD,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,1,Dimanche,1,855,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,1,Dimanche,1,1056,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,1,Dimanche,1,1057,Entity2 Belgique,Administrative et Financière,Finances PCX,NaN,NaN,2010-01-01 00:00:00.000,NaN,NaN,1800,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,1,Dimanche,1,1587,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,15,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,1,Dimanche,1,1589,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,20,0,0,0,0,0,0,0,0
5,2011-04-24 01:30:00.000,1,Dimanche,1,1591,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,10800,832,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0
6,2011-04-24 01:30:00.000,1,Dimanche,1,1555,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,NaN,NaN,12600,2632,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,10,10,1,0,7,0,0,1,15,0,0,0,1,0,1,0,488
7,2011-04-25 01:30:00.000,0,Lundi,1,21,Entity1 France,Commerciale,RTC,Relation Téléphonique Clientèle,CLIENTS,2001-01-01 00:00:00.000,NaN,NaN,1800,0,0,0,0,0,0,0,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20,0,0,0,0,0,0,0,0
8,2011-04-25 01:30:00.000,0,Lundi,1,711,Entity1 France,Administrative et Financière,Gestion Renault,Gestion,ADMINISTRATIF,2009-01-01 00:00:00.000,NaN,NaN,7200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,30,0,0,0,0,0,0,0,0
9,2011-04-25 01:30:00.000,0,Lundi,1,20,Entity1 France,Commerciale,RTC,Relation Téléphonique Clientèle,CLIENTS,2001-01-01 00:00:00.000,NaN,NaN,1800,0,0,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20,0,0,0,0,0,0,0,0


## View main statistics

In [5]:
training_data.describe()

,WEEK_END,TPER_HOUR,SPLIT_COD,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,0.293060,1.20600,732.078300,5612.639110,2147.192230,19.188360,142.018240,8.881250,1.152800,8.618060,2.240030,2053.997620,0.013560,1.199150,0.008890,2.439150,0.087930,8.972090,0.079500,7.943990,0.089370,8.843660,0.076730,8.526670,0.004110,0.110760,1.196920,0.024200,1.482590,0.002010,0.009910,0.005360,0.004040,0.252290,0.001680,0.000480,0.000150,0.000120,0.000070,0.000090,0.00021,0.000080,0.000120,0.001040,0.004480,0.058860,0.055720,0.122470,0.653810,0.659230,0.112790,0.000200,3.133600,0.003190,0.377300,0.105600,22.681800,0.005980,0.002490,0.003940,0.113120,0.002360,0.114800,25.46101,19.789930
std,0.455168,1.00576,434.627587,6932.612088,2781.807498,139.675121,624.897012,79.140042,24.645588,70.209562,49.517911,3687.802242,0.220537,36.144933,0.128262,87.977185,0.810756,81.116774,0.765144,73.503489,0.655643,78.439583,0.602035,77.105594,0.083864,0.725105,16.649945,0.200686,17.739521,0.051439,0.110236,0.086089,0.093722,13.125648,0.045576,0.022799,0.012247,0.010954,0.008366,0.009486,0.01581,0.008944,0.010954,0.047528,0.081731,2.532619,0.233443,0.756014,4.089578,4.130893,0.735115,0.014141,3.854172,0.077717,14.686642,0.698279,10.445001,0.096147,0.057999,0.076187,0.736715,0.077165,0.746222,295.12391,145.176728
min,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.00000,413.000000,1800.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

##  A simple predictor

Lets try building a tree-based boosting predictor with very few attributes just to see how it goes.
This predictor will only predict the number of calls received during a given time stamp (e.g. in a 30 minutes slot) and day of week.

Note that the date is not relevant for regression, but we can extract some relevant information from it: day of the week, time slot, and if it is a week-end or not.

Also, for some reason the data for a given ASS_ASSIGNMENT and DATE is sometimes split, so we have to aggregate it.

In [6]:
#Select only basic attributes

#Extract the time slot from date
def get_time(date):
    time = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').time()
    return time.hour * 3600 + time.minute * 60 + time.second

#Assign a number to the day of the week
day_to_num_dict = {j:i for i,j in enumerate(['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche'])}

training_data['TIME'] = training_data['DATE'].map(get_time)
training_data['DAY'] = training_data['DAY_WE_DS'].map(day_to_num_dict)
training_data_simple = training_data[['DATE','TIME','ASS_ASSIGNMENT','DAY','WEEK_END','CSPL_RECEIVED_CALLS']]

#Aggregate the calls
grouped = training_data_simple.groupby(['DATE','TIME','WEEK_END','DAY','ASS_ASSIGNMENT']).sum().reset_index()

#Now that we aggregated the calls, the date is not relevant (only the time slot is)
grouped = grouped.drop('DATE',axis=1)
grouped.head(n=20)

#Convert the different ASS_ASSIGNMENTs to booleans
for value in grouped.ASS_ASSIGNMENT.unique():
        grouped["ASS_ASSIGNMENT_"+value] = (grouped.ASS_ASSIGNMENT == value)

#Extract features in X, target in y
cols = [col for col in grouped.columns if col not in ['CSPL_RECEIVED_CALLS', 'ASS_ASSIGNMENT']]
X = grouped[cols]
y = grouped['CSPL_RECEIVED_CALLS']

X.head(n=20)

,TIME,WEEK_END,DAY,ASS_ASSIGNMENT_Crises,ASS_ASSIGNMENT_DOMISERVE,ASS_ASSIGNMENT_Domicile,ASS_ASSIGNMENT_Gestion,ASS_ASSIGNMENT_Gestion - Accueil Telephonique,ASS_ASSIGNMENT_Gestion Amex,ASS_ASSIGNMENT_Gestion Assurances,ASS_ASSIGNMENT_Gestion Clients,ASS_ASSIGNMENT_Gestion Renault,ASS_ASSIGNMENT_Japon,ASS_ASSIGNMENT_LifeStyle,ASS_ASSIGNMENT_Manager,ASS_ASSIGNMENT_Maroc - Génériques,ASS_ASSIGNMENT_Medicine,ASS_ASSIGNMENT_Médical,ASS_ASSIGNMENT_NL Technique,ASS_ASSIGNMENT_Nuit,ASS_ASSIGNMENT_RENAULT,ASS_ASSIGNMENT_Regulation Medicale,ASS_ASSIGNMENT_Réception,ASS_ASSIGNMENT_SAP,ASS_ASSIGNMENT_Services,ASS_ASSIGNMENT_TAI - CARTES,ASS_ASSIGNMENT_TAI - PNEUMATIQUES,ASS_ASSIGNMENT_TAI - RISQUE,ASS_ASSIGNMENT_TAI - RISQUE SERVICES,ASS_ASSIGNMENT_TAI - SERVICE,ASS_ASSIGNMENT_TPA,ASS_ASSIGNMENT_Tech. Axa,ASS_ASSIGNMENT_Tech. Inter,ASS_ASSIGNMENT_Technical,ASS_ASSIGNMENT_Technique Belgique,ASS_ASSIGNMENT_Technique International,ASS_ASSIGNMENT_Truck Assistance,ASS_ASSIGNMENT_Téléphonie,ASS_ASSIGNMENT_Tech. Total,ASS_ASSIGNMENT_CAT,ASS_ASSIGNMENT_Gestion DZ,ASS_ASSIGNMENT_NL Médical,ASS_ASSIGNMENT_Finances PCX,ASS_ASSIGNMENT_Mécanicien,ASS_ASSIGNMENT_TAI - PANNE MECANIQUE,ASS_ASSIGNMENT_RTC,ASS_ASSIGNMENT_CMS,ASS_ASSIGNMENT_Maroc - Renault,ASS_ASSIGNMENT_Gestion Relation Clienteles,ASS_ASSIGNMENT_FO Remboursement,ASS_ASSIGNMENT_Divers,ASS_ASSIGNMENT_Prestataires,ASS_ASSIGNMENT_AEVA,ASS_ASSIGNMENT_Evenements
0,0,1,5,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,1,5,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,1,5,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,1,5,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0,1,5,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,0,1,5,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,0,1,5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,0,1,5,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,0,1,5,False,False,False,False,False,False,False,False,True,False,False,Fals

In [25]:
#Now create the gradient boosting regressor
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(__version__)
est = GradientBoostingRegressor()

tuned_parameters = {'n_estimators':[20,30,50],'learning_rate': [0.1,0.3,0.5], 'subsample': [1.0,0.7,0.5],
                     'min_samples_split':[1],'min_samples_leaf':[1,2],
                     'max_depth':[1,3,5,8],'max_features':['auto','sqrt','log2']
                    }
                     

clf = RandomizedSearchCV(est, tuned_parameters, cv=5,n_jobs=-1,n_iter=10)

clf.fit(X_train, y_train)



#Plot CV error (this is squared loss, which will be used to evaluate our performance in the leaderboard)
print(clf.best_params_)

0.17.1
{'n_estimators': 20, 'learning_rate': 0.5, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 1, 'subsample': 1.0, 'max_features': 'auto'}


TODO: Use sklearn.grid_search.GridSearchCV to find the best set of parameters.
TODO: Write code to make a first submission to the leaderboard

# Prediction and submission

In [8]:
submission = pd.read_csv("data/submission.txt", sep='\t')
X = submission.copy()
submission.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-01-03 00:00:00.000,CAT,0
1,2012-01-03 00:00:00.000,Téléphonie,0
2,2012-01-03 00:00:00.000,Tech. Inter,0
3,2012-01-03 00:00:00.000,Tech. Axa,0
4,2012-01-03 00:00:00.000,Services,0


In [9]:
def get_weekday(date):
    return dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').weekday()

X['TIME'] = X.DATE.map(get_time)
X['DAY'] = X.DATE.map(get_weekday)
X['WEEK_END'] = X.DAY.isin([5, 6]) * 1

#Convert the different ASS_ASSIGNMENTs to booleans
for value in grouped.ASS_ASSIGNMENT.unique():
    X["ASS_ASSIGNMENT_"+value] = (X.ASS_ASSIGNMENT == value)

In [10]:
X = X[cols]
submission.prediction = est.predict(X)
submission.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-01-03 00:00:00.000,CAT,0.065884
1,2012-01-03 00:00:00.000,Téléphonie,0.781253
2,2012-01-03 00:00:00.000,Tech. Inter,0.065884
3,2012-01-03 00:00:00.000,Tech. Axa,0.065884
4,2012-01-03 00:00:00.000,Services,0.229027


In [11]:
# Write prediction to csv
submission.to_csv("data/output.txt", sep='\t', index=False)